In [747]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns

from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os
from sklearn.cluster import KMeans
import itertools

Populating the interactive namespace from numpy and matplotlib


In [661]:
hdfsclient = Client("amp-spark-master.amp", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [662]:
executor_mem = "200g"
cores_per_executor = 32
num_executors = 14
EGR1_PATH = '/user/vaishaal/endive-data/EGR1'
ATF3_PATH = '/user/vaishaal/endive-data/ATF3'

In [700]:
W = run_filter_generation("/user/akmorrow/full_dnase/EGR1", "/tmp/filters.csv")

In [ ]:
W

In [ ]:
# this is where the magic happens
results = [] 
num_filters = [256]
gammas = [1.0]
dnase_gammas = [1.0]
kmer_sizes = [8]
dnase_kmer_sizes = [100]
replicates = [0]
normalizeDnases = [False]

params = list(itertools.product(num_filters, gammas, dnase_gammas, kmer_sizes, dnase_kmer_sizes, replicates, normalizeDnases))

In [ ]:
params

In [ ]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])
for param in params:
    num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate, normalizeDnase = param
    feat_out_loc = "/user/vaishaal/featurizedWindows/empirical-sequence-dnase-normalized-{6}/tiny/{0}-filters/{1}-gamma/{2}-dnase-gamma/{3}-kmers/{4}-dnase-kmers/replicate-{5}"\
                    .format(num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate, normalizeDnase)
    res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           W,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           cores_per_executor=cores_per_executor,
                           sample=0.01,
                           alphabet_size=4,
                           kmer_size=kmer_size,
                           dnase_kmer_size=dnase_kmer_size,
                           gamma=gamma,
                           dnase_gamma=dnase_gamma,
                           executor_mem=executor_mem,
                           num_filters=num_filter,
                           normalizeDnase=normalizeDnase,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput=feat_out_loc,
                           num_executors=num_executors)
                

W SHAPE  (256, 232)
NORMALIZE DNASE  False


In [ ]:
results_dfs = []

In [773]:
for param in params:
    num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate, normalizeDnase = param
    feat_out_loc = "/user/vaishaal/featurizedWindows/sequence-dnase-normalized-{6}/tiny/{0}-filters/{1}-gamma/{2}-dnase-gamma/{3}-kmers/{4}-dnase-kmers/replicate-{5}"\
                    .format(num_filter, gamma, dnase_gamma, kmer_size, dnase_kmer_size, replicate, normalizeDnase)
    print("FEAT OUT LOC " + feat_out_loc)
    meta = {}
    meta['num_filter'] = num_filter
    meta['dnase_kmer_size'] = dnase_kmer_size
    meta['dnase_gamma'] = dnase_gamma
    meta['gamma'] = gamma
    meta['cutoff'] = None
    for hn in [1]:
        results_df, train, val = cross_validate(feat_out_loc, 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            numHoldOutChr=3, 
                            numHoldOutCell=1,
                            cores_per_executor = 32,
                            num_executors = 14,
                            executor_mem="150g",
                            num_folds=1, 
                            negativeSamplingFreqs=[0.01],
                            numItersHardNegative=hn,
                            regs=[1e-8],
                            other_meta=meta)
        print(results_df)
        results_dfs.append(results_df)    

FEAT OUT LOC /user/vaishaal/featurizedWindows/sequence-dnase-normalized-False/tiny/256-filters/1.0-gamma/1.0-dnase-gamma/8-kmers/100-dnase-kmers/replicate-0
FOLD 0
HOLDING OUT CHROMOSOMES ['chr12', 'chr11', 'chr22']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=1e-08

(2649302,)
(-1, 2)

(54010,)
(-1, 2)
COMPUTING train metrics
COMPUTING val metrics
  cutoff  dnase_gamma  dnase_kmer_size  gamma  negativeSamplingFreq  \
0   None            1              100      1                  0.01   

   num_filter  num_iters_hard_negative           reg test_celltypes  \
0         256                        1  1.000000e-08            [3]   

        test_chromosomes          ...           train_recall_at_05_fdr  \
0  [chr12, chr11, chr22]          ...                         0.003208   

   train_recall_at_10_fdr  train_recall_at_25_fdr  train_recall_at_50_fdr  \
0                0.009624                0.098132                0.764484   

   val_auPRC  val_auROC  val_recall_at_05_fdr  val_rec


##### results_dfs[-1]

In [739]:
results_dfs[-1]

,cutoff,dnase_gamma,dnase_kmer_size,gamma,negativeSamplingFreq,num_filter,num_iters_hard_negative,reg,test_celltypes,test_chromosomes,...,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,None,1,100,1,0.1,256,1,1.000000e-08,[3],"[chr12, chr11, chr22]",...,0,0,0.004676,0.241367,0.195153,0.983943,0.074074,0.074074,0.074074,0.074074


### pd.concat(results_dfs)[results_dfs[0].columns[2:]]

In [ ]:
val_copy = val[0].copy()[np.where(val[0][:, 0] >= 0), :].reshape(-1, 3)

In [ ]:
old_results = pd.concat(results_dfs)

In [ ]:
print(old_results.to_csv())

In [ ]:
val_copy[np.where(val_copy[:, 0] == -1)]


In [ ]:
results_dfs[-1]

In [ ]:
print val[0][np.argsort(val[0][:, 1]), 2]

In [ ]:
sns.stripplot(y="val_auROC", x="negativeSamplingFreq", hue="num_iters_hard_negative", data=pd.concat(results_dfs))

In [ ]:
pd.concat(results_dfs)[results_dfs[0].columns[2:]]

In [ ]:
print(pd.concat(results_dfs).to_csv())

In [ ]:
all_res = pd.concat(results_dfs)

In [ ]:
res = pd.concat(results_dfs)[results_dfs[0].columns[2:]][all_res['num_iters_hard_negative'] == 1]

In [ ]:
sns.stripplot(data=res, y="val_auPRC", x="num_filter")

In [ ]:
old_results = results_dfs

In [ ]:
params